# Neural Prophet
### Kaggle 
score = 0.410

In [1]:
import pandas as pd
from neuralprophet import NeuralProphet

# 1️⃣ Cargar los datasets
ventas = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
ventas['periodo_dt'] = pd.to_datetime(ventas['periodo'].astype(str), format='%Y%m')





# Unir con ventas reales
ventas = ventas[['product_id','periodo_dt','tn']]
ventas['tn'] = ventas['tn'].fillna(0)
ventas['ds'] = ventas['periodo_dt']
ventas['y'] = ventas['tn']

# Filtrar solo productos a predecir
productos_ids = ventas['product_id'].unique()

# 2️⃣ Entrenar un modelo por producto a predecir
resultados = []
fecha_prediccion = pd.to_datetime('2020-02-01')
errores = []
for pid in productos_ids:
    df_prod = ventas[ventas['product_id']==pid][['ds','y']].sort_values('ds')
    
    if len(df_prod) < 3:
        print(f"⏩ Saltando producto {pid}: muy pocos datos.")
        errores.append({'product_id': pid, 'periodo': 202002, 'tn_pred': None})
        continue
    
    m = NeuralProphet(n_lags=12, n_forecasts=2, epochs=100, yearly_seasonality=True)
    try:
        m.fit(df_prod, freq='MS', progress='bar')
        future = m.make_future_dataframe(df_prod, periods=2, n_historic_predictions=False)
        forecast = m.predict(future)
        resultados.append(forecast)
    except Exception as e:
        print(f"❌ Error en producto {pid}: {e}")
        errores.append({'product_id': pid, 'periodo': 202002, 'tn_pred': None})

# 3️⃣ Guardar resultados
df_result = pd.DataFrame(resultados)

ModuleNotFoundError: No module named 'neuralprophet'

In [ ]:
df_result['tn_pred'] = df_result['tn_pred'].fillna(0)
df_result['tn_pred'] = df_result['tn_pred'].apply(lambda x: 0 if x < 0 else x)

In [4]:
df_result

,product_id,periodo,tn_pred
0,20001,202002,1282.631958
1,20002,202002,1019.728638
2,20003,202002,850.583252
3,20004,202002,428.728302
4,20005,202002,639.941406
...,...,...,...
775,21263,202002,0.000000
776,21265,202002,0.000000
777,21266,202002,0.000000
778,21267,202002,0.000000


In [5]:
df_result.rename(columns={'tn_pred': 'tn'}, inplace=True)
df_result.drop(columns=['periodo'], inplace=True)

In [8]:
df_result.to_csv('./kaggle/neuralprophet_feb2020.csv', index=False)